The `get_youtube_transcription` function attempts to retrieve subtitles from a YouTube video URL in any available language, preferring manually created subtitles over auto-generated ones.

**If no subtitles are available**, the function downloads the audio from the YouTube video and uses OpenAI's Whisper model to transcribe it.


In [ ]:
# Install required libraries
!pip install yt-dlp
#!pip install whisper
!pip install git+https://github.com/openai/whisper.git
!pip install openai
!pip install youtube_transcript_api

import os
import yt_dlp
import whisper
import openai
import time

In [44]:
def get_youtube_transcription(url: str) -> str:
    """
    Check if YouTube has subtitles available for a given URL and download them in any available language.

    Parameters:
    - url (str): The URL of the YouTube video.

    Returns:
    str: The subtitles if available, otherwise None.
    """
    from youtube_transcript_api import YouTubeTranscriptApi
    from youtube_transcript_api._errors import (
        NoTranscriptFound,
        TranscriptsDisabled,
        VideoUnavailable,
    )
    import urllib.parse

    # Helper function to extract video ID
    def extract_video_id(url):
        # Parse URL
        parsed_url = urllib.parse.urlparse(url)
        hostname = parsed_url.hostname.lower() if parsed_url.hostname else ''
        if 'youtu.be' in hostname:
            # Shortened URL format
            return parsed_url.path[1:]
        elif 'youtube.com' in hostname:
            if parsed_url.path == '/watch':
                # URL with 'v' parameter
                query = urllib.parse.parse_qs(parsed_url.query)
                return query.get('v', [None])[0]
                return query.get('v', [None])[0]
            elif parsed_url.path.startswith(('/embed/', '/v/')):
                return parsed_url.path.split('/')[2]
        return None

    video_id = extract_video_id(url)
    if not video_id:
        return None  # Could not extract video ID

    try:
        # Get list of available transcripts
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

        # Separate manually created and generated transcripts
        manually_created_transcripts = []
        generated_transcripts = []

        for transcript in transcript_list:
            if not transcript.is_generated:
                manually_created_transcripts.append(transcript)
            else:
                generated_transcripts.append(transcript)

        # Try manually created transcripts first
        for transcript in manually_created_transcripts:
            try:
                transcript_data = transcript.fetch()
                transcript_text = ' '.join([t['text'] for t in transcript_data])
                return transcript_text
            except Exception:
                continue  # Try next transcript

        # If no manually created transcripts, try generated transcripts
        for transcript in generated_transcripts:
            try:
                transcript_data = transcript.fetch()
                transcript_text = ' '.join([t['text'] for t in transcript_data])
                return transcript_text
            except Exception:
                continue  # Try next transcript

        # No transcripts found
        return None
    except (NoTranscriptFound, TranscriptsDisabled, VideoUnavailable):
        # Subtitles not available or video is unavailable
        return None
    except Exception as e:
        # Other exceptions
        print(f"An error occurred: {e}")
        return None


In [45]:
# Function to transcribe audio from YouTube URL
def audio_transcriber_tool(url: str) -> str:
    """
    Extracts audio and transcribes it from a YouTube video given its URL.

    Parameters:
    - url (str): The URL of the YouTube video.

    Returns:
    str: The transcribed text from the YouTube audio.
    """
    try:
        # Use yt-dlp to download the audio from the YouTube video
        ydl_opts = {
            'format': 'bestaudio/best',  # Download the best available audio
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',  # Extract as mp3
                'preferredquality': '192',  # Audio quality
            }],
            'outtmpl': 'audio_file.%(ext)s',  # Output filename with extension
            'quiet': True,  # Suppress console output
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])  # Download the audio from the URL

        # Find the downloaded audio file (assuming mp3 extension)
        audio_file = "audio_file.mp3"

        # Load the Whisper model (small model for faster transcription)
        whisper_model = whisper.load_model("small")

        # Transcribe the downloaded audio file
        result = whisper_model.transcribe(audio_file)

        # Clean up the audio file after transcription
        os.remove(audio_file)

        return result["text"]
    except Exception as e:
        return f"Error downloading or transcribing audio: {e}"

# Function to process podcast video using OpenAI (if needed)
def process_transcription(transcription: str, openai_api_key: str):
    """
    Uses OpenAI GPT model to summarize the transcription and extract insights.

    Parameters:
    - transcription (str): The transcription text from Whisper.
    - openai_api_key (str): The OpenAI API key.

    Returns:
    str: The summarized output.
    """
    openai.api_key = openai_api_key
    response = openai.Completion.create(
      model="gpt-3.5-turbo",
      prompt=f"Summarize this podcast transcription:\n{transcription}",
      temperature=0.5,
      max_tokens=500
    )
    return response.choices[0].text.strip()

In [ ]:
# Optional for debugging purposes, checking whether the video has subtitles available
!yt-dlp --list-subs https://www.youtube.com/watch?v=5h0iq1NqpCo

In [46]:
# Example Usage
youtube_url = input("Enter the YouTube URL of the podcast you want to analyze: ")
#youtube_url = "https://www.youtube.com/watch?v=zJgHbifIx-Q"
#youtube_url = "https://www.youtube.com/watch?v=5h0iq1NqpCo"

# Step 1: Check if subtitles are available
print("Checking for available subtitles...")
transcription = get_youtube_transcription(youtube_url)

if transcription:
    print("Using YouTube subtitles for transcription.")
else:
    print("No subtitles found, using Whisper for transcription.")

Checking for available subtitles...
Using YouTube subtitles for transcription.


In [ ]:
# Step 2: If no subtitles, fall back to Whisper transcription
if not transcription:
    start_time = time.time()
    transcription = audio_transcriber_tool(youtube_url)

    elapsed_time = time.time() - start_time
    print(f"Transcription completed in {elapsed_time:.2f} seconds.\n")

# Display the transcription
print("Transcription:\n")
print(transcription)

In [ ]:
# Step 2: Process transcription with OpenAI (if you want summarization)
openai_api_key = input("Enter your OpenAI API key: ")

print("\nProcessing transcription with OpenAI...\n")
summary = process_transcription(transcription, openai_api_key)

# Display the summary
print("Summary:\n")
print(summary)